In [1]:
import json

import boto3

In [2]:
# Defined at ~/.aws/credentials
AWS_PROFILE = 'demo'

AWS_REGION = 'us-east-1'

BUCKET = 's3-select-filtering-demo'

KEY = 'users.csv'

In [3]:
session = boto3.Session(profile_name=AWS_PROFILE, region_name=AWS_REGION)

s3 = session.resource('s3')

# Write on S3

In [28]:
!cat ../data/users.csv

id,email,name,lastname,birthdate,created_at
1,john@test.com,john,doe,1980-01-01T,2023-01-01T15:00:00
2,jane@test.com,jane,doe,1980-01-01T,2023-01-01T03:00:00
3,johnsmith@test.com,john,smith,1990-01-01T,2023-01-01T11:00:00
4,janesmith@test.com,jane,smith,1990-01-01T,2023-01-01T12:00:00
5,michael@test.com,michael,smith,2022-01-01T,2023-01-15T01:00:00

In [62]:
res = s3.Object(BUCKET, KEY).put(Body=open(f'../data/{KEY}', 'rb'))

# Filter data in S3

In [6]:
def execute_query(bucket, key, query, output_type='JSON'):
    
    res = s3.meta.client.select_object_content(
        Bucket=bucket, 
        Key=key,
        ExpressionType='SQL',
        Expression=query,
        InputSerialization={'CSV': {"FileHeaderInfo": "Use"}, 'CompressionType': 'NONE'},
        OutputSerialization={output_type: {}},
    )
    records = []
    for event in res['Payload']: # Payload is an EventStream
        if 'Records' in event:
            if output_type == 'JSON':
                records.extend([json.loads(e) for e in event['Records']['Payload'].decode('utf-8').split()])
            else:
                records.extend([e for e in event['Records']['Payload'].decode('utf-8').split()])
    return records

# Execute SQL Query

In [75]:
QUERY = "SELECT s.email, s.created_at FROM s3object s WHERE s.created_at > '2022-10-01'"

### As `JSON`

In [76]:
%%time
execute_query(bucket=BUCKET, key=KEY, query=QUERY, output_type='JSON')

CPU times: user 18.6 ms, sys: 0 ns, total: 18.6 ms
Wall time: 642 ms


[{'email': 'janesmith@test.com', 'created_at': '2022-12-01T12:00:00'},
 {'email': 'michael@test.com', 'created_at': '2023-01-15T01:00:00'}]

### As `CSV` 

In [77]:
execute_query(bucket=BUCKET, key=KEY, query=QUERY, output_type='CSV')

['janesmith@test.com,2022-12-01T12:00:00',
 'michael@test.com,2023-01-15T01:00:00']